# **Credit Card Customer churn Analysis - Load**

## Objectives

* By the end of the load phase, I will:
    1. Summarize ETL steps
    2. Outline the next steps
    3. Prepare the file for further analysis 

## Inputs
* Extract and transform phase outcomes
* bank_churners_transformed_v2.csv


## Outputs

* Next steps



---

importing libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
import scipy
import scipy.stats as stats
import plotly.express as px
from scipy.stats import mannwhitneyu

# Data load and sanity check

Reloading the data and checking missing data and duplicates

In [3]:
df = pd.read_csv("../data/bank_churners_transformed_v2.csv") #Loading the new dataset
df

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,mod_Gender,mod_Education_Level,mod_Income_Category,mod_Card_Category,handled_Total_Trans_Amt,mod_Age_Group,Trans_per_Month,Avg_Trans_Amt,Util_Ratio,Attrition_Label
0,768805383,Existing Customer,45,M,3,High School,2.0,$60K - $80K,Blue,39,...,1,1.0,3.0,0,7.043160,Adults,1.076923,27.238095,0.061224,Retained
1,818770008,Existing Customer,49,F,5,Graduate,1.0,Less than $40K,Blue,44,...,0,3.0,1.0,0,7.163947,Adults,0.750000,39.121212,0.104651,Retained
2,713982108,Existing Customer,51,M,3,Graduate,2.0,$80K - $120K,Blue,36,...,1,3.0,4.0,0,7.543273,Adults,0.555556,94.350000,0.000000,Retained
3,769911858,Existing Customer,40,F,4,High School,NaN,Less than $40K,Blue,34,...,0,1.0,1.0,0,7.066467,Adults,0.588235,58.550000,0.759734,Retained
4,709106358,Existing Customer,40,M,3,Uneducated,2.0,$60K - $80K,Blue,21,...,1,0.0,3.0,0,6.705639,Adults,1.333333,29.142857,0.000000,Retained
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,772366833,Existing Customer,50,M,2,Graduate,1.0,$40K - $60K,Blue,40,...,1,3.0,2.0,0,9.647110,Adults,2.925000,132.273504,0.462403,Retained
10123,710638233,Attrited Customer,41,M,2,Unknown,3.0,$40K - $60K,Blue,25,...,1,NaN,2.0,0,9.078522,Adults,2.760000,127.014493,0.511106,Attrited
10124,716506083,Attrited Customer,44,F,1,High School,2.0,Less than $40K,Blue,36,...,0,1.0,1.0,0,9.239122,Adults,1.666667,171.516667,0.000000,Attrited
10125,717406983,Attrited Customer,30,M,2,Graduate,NaN,$40K - $60K,Blue,36,...,1,3.0,2.0,0,9.035511,Young Adults,1.722222,135.403226,0.000000,Attrited


In [6]:
df.isnull().sum() #Checking for missing values

CLIENTNUM                      0
Attrition_Flag                 0
Customer_Age                   0
Gender                         0
Dependent_count                0
Education_Level                0
Marital_Status               749
Income_Category                0
Card_Category                  0
Months_on_book                 0
Total_Relationship_Count       0
Months_Inactive_12_mon         0
Contacts_Count_12_mon          0
Credit_Limit                   0
Total_Revolving_Bal            0
Avg_Open_To_Buy                0
Total_Amt_Chng_Q4_Q1           0
Total_Trans_Amt                0
Total_Trans_Ct                 0
Total_Ct_Chng_Q4_Q1            0
Avg_Utilization_Ratio          0
mod_Attrition_Flag             0
mod_Gender                     0
mod_Education_Level         1519
mod_Income_Category         1112
mod_Card_Category              0
handled_Total_Trans_Amt        0
mod_Age_Group                  0
Trans_per_Month                0
Avg_Trans_Amt                  0
Util_Ratio

**Heads-up**: These 0s introduced are part of the encoding process. They are intenational, represent cardinal(e.g., card categories)/ordinal values(e.g., education levels), and not errors.

In [7]:
df[df.duplicated()] #Checking for duplicates

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,mod_Gender,mod_Education_Level,mod_Income_Category,mod_Card_Category,handled_Total_Trans_Amt,mod_Age_Group,Trans_per_Month,Avg_Trans_Amt,Util_Ratio,Attrition_Label


I have not duplicated values as part of the transform phase.

---

# Column descriptions

## Variable Definitions ##

The following table provides definitions and descriptions for all variables in the credit card customer dataset:

| **#** | **Variable Name** | **Description** | **Data Type** | **Example Values** |
|-------|-------------------|-----------------|---------------|-------------------|
| 1 | **CLIENTNUM** | Unique client/customer number (identifier) | Integer | 768805383, 818770008 |
| 2 | **Attrition_Flag** | Customer activity status | Categorical | "Attrited Customer", "Existing Customer" |
| 3 | **Customer_Age** | Customer's age in years | Integer | 26-73 years |
| 4 | **Gender** | Customer's gender | Categorical | "M" (Male), "F" (Female) |
| 5 | **Dependent_count** | Number of dependents financially dependent on customer | Integer | 0-5 dependents |
| 6 | **Education_Level** | Educational qualification | Categorical | "High School", "Graduate", "Doctorate", etc. |
| 7 | **Marital_Status** | Marital status | Categorical | "Married", "Single", "Divorced", "Unknown" |
| 8 | **Income_Category** | Annual income range | Categorical | "Less than $40K", "$40K-$60K", "$60K-$80K", etc. |
| 9 | **Card_Category** | Type of credit card | Categorical | "Blue", "Silver", "Gold", "Platinum" |
| 10 | **Months_on_book** | Period of relationship with bank (in months) | Integer | 13-56 months |
| 11 | **Total_Relationship_Count** | Number of products held by customer with the bank | Integer | 1-6 products |
| 12 | **Months_Inactive_12_mon** | Number of months inactive in the last 12 months | Integer | 0-6 months |
| 13 | **Contacts_Count_12_mon** | Number of contacts with the bank in the last 12 months | Integer | 0-6 contacts |
| 14 | **Credit_Limit** | Credit limit on the card | Float | $1,438 - $34,516 |
| 15 | **Total_Revolving_Bal** | Total revolving balance (outstanding balance not paid off) | Float | $0 - $2,517 |
| 16 | **Avg_Open_To_Buy** | Average open-to-buy credit line (credit limit minus current balance) | Float | Calculated field |
| 17 | **Total_Amt_Chng_Q4_Q1** | Change in transaction amount (Q4 over Q1) | Float | Ratio/percentage change |
| 18 | **Total_Trans_Amt** | Total transaction amount (last 12 months) | Float | Annual transaction volume |
| 19 | **Total_Trans_Ct** | Total transaction count (last 12 months) | Integer | Annual transaction frequency |
| 20 | **Total_Ct_Chng_Q4_Q1** | Change in transaction count (Q4 over Q1) | Float | Ratio/percentage change |
| 21 | **Avg_Utilization_Ratio** | Average card utilization ratio (balance to credit limit) | Float | 0.0 - 1.0 (0% - 100%) |

## Transformed Variables: Legend 

The following table shows how categorical variables were transformed into numerical values for analysis:

**Variable Encoding Reference:**

| **Variable** | **Original Category** | **Encoded Value** | **Description** |
|--------------|----------------------|-------------------|-----------------|
| **mod_Attrition_Flag** | Existing Customer | 0 | Customer retained |
| | Attrited Customer | 1 | Customer churned |
| **mod_Gender** | F (Female) | 0 | Female customer |
| | M (Male) | 1 | Male customer |
| **mod_Education_Level** | Unknown | NaN | Education level unknown |
| | Uneducated | 1 | No formal education |
| | High School | 2 | High school graduate |
| | College | 3 | College graduate |
| | Graduate | 4 | Graduate degree |
| | Post-Graduate | 5 | Post-graduate degree |
| | Doctorate | 6 | Doctorate degree |
| **Marital_Status** | Unknown | NaN | Marital status unknown |
| | Single | 1 | Single |
| | Married | 2 | Married |
| | Divorced | 3 | Divorced |
| **mod_Income_Category** | Unknown | NaN | Income unknown |
| | Less than $40K | 1 | Low income |
| | $40K - $60K | 2 | Lower-middle income |
| | $60K - $80K | 3 | Middle income |
| | $80K - $120K | 4 | Upper-middle income |
| | $120K + | 5 | High income |
| **mod_Card_Category** | Blue | 0 | Basic card tier |
| | Silver | 1 | Silver card tier |
| | Gold | 2 | Gold card tier |
| | Platinum | 3 | Premium card tier |

## Engineered Features

| Feature                      | Formula                                                   | Why It Helps                                                                                                           |
| ---------------------------- | --------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------- |
| **Transactions per month**   | `Total_Trans_Ct / Months_on_book`                         | Captures average engagement — more active customers are less likely to churn.                                          |
| **Avg transaction amount**   | `Total_Trans_Amt / Total_Trans_Ct`                        | Signals spending behavior per transaction — useful to differentiate low-frequency big spenders vs frequent small ones. |
| **Utilization ratio**        | `Total_Revolving_Bal / Credit_Limit`                      | Standard credit card KPI — may indicate financial stress or dependency.                                                |

---

# Assumptions

1. Discoveries that require further analysis can be picked up in the next sprint. For instance, some variables require the 0s to be handled. This will require SME inputs and detailed analysis and so, I've decided to push it out to the next sprint.

2. ML pipeline creation will follow this stage and the jupyter notebooks can be leveraged for that.

3. Business impact and recommendations require a round of review before being presented to the final stakeholders.

---

# ETL Summary

**Extract:** 
    
    a. Loaded the credit card customer dataset from Kaggle.

    b. Reviewed dataset structure including variables, missing values, and duplicates and captured assumptions.

    c. Generated initial visualizations to understand distributions and patterns and performed an initial correlation analysis.

    d. Saved the newly generated file for transformation

**Transform:** 

    a. Encoded categorical variables for correlation analysis and statistical test.

    b. Validated hypotheses with statistical tests and discovered churn predictors.

    c. Plotted graphs to visualize validated results and captured business impact.

    d. Saved the transformed dataset for further analysis.

**Load:**

    a. Reloded the transformed dataset and performed a sanity check.

    b. Captured assumptions and next steps.

---

# Next steps

1. Build the features identified in the transform stage.

2. Identify and handle outliers.

3. Discuss the validity of 0 values in the variables (for e.g., Total_Revolving_Balance).

4. Build a dashboard with the existing findings and set the stage for further discussion.

5. The bank_churners_transformed_v2.csv is ready for further analysis.
